## 1. Exploratory Data Analysis (EDA)

Techniques: Histograms, PDF overlay, KDE, boxplots, correlation heatmaps, pairplots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import norm

sns.set()

In [ ]:
# Specify the dataset
data = r'C:\Users\Student\Downloads\HTL_data.csv'
df = pd.read_csv(data)

In [ ]:
# Read the first five entries of the dataset
df.head()

In [ ]:
# Obtain the statistics about the data
df.describe()

In [ ]:
# Obtain information about the variable types
df.info()

In [ ]:
# Define the features
df.columns = df.columns.str.strip().str.lower()
features = ['bwr', 'temp', 'time', 'pressure']

# Ensure that none of the columns should be missed to be read properly
missing = [col for col in features if col not in df.columns]
print("Missing columns:", missing)

In [ ]:
# Plotting histograms
sns.histplot(df.temp)

In [ ]:
# Histogram plot along with the PDF and KDE
x = np.linspace(min(df.temp), max(df.temp), 1000)
mu = np.mean(df.temp)
sigma = np.std(df.temp)

y = norm.pdf(x, mu, sigma)

plt.plot(x, y, color="red", label="Gaussian PDF")
sns.histplot(df.temp, kde=True, stat="density", color="blue", label="Histogram+KDE")

# Computing the bandwidth using Scott's Rule
n = len(df.temp)
h_scott = n**(-1/5) * sigma
sns.kdeplot(df.temp, bw_method=h_scott*0.03, color="green", label="KDE")
plt.legend()

In [ ]:
# Density plots for all features
df[features].plot(kind="density", subplots=True)

In [ ]:
# Univariate boxplot with outlier detection
sns.boxplot(df.temp)

# Manual Evaluation Of the boxplot
Q1 = np.percentile(df.temp, 25)
Q3 = np.percentile(df.temp, 75)
IQR = Q3 - Q1

# Defining the lb and ub
lb = Q1 - 1.5*IQR
ub = Q3 + 1.5*IQR
outlier = df.temp[(df.temp < lb) | (df.temp > ub)]
print(outlier)

In [ ]:
# Multivariate boxplot
sns.boxplot(df)

In [ ]:
# Correlation Matrix
corr_mat = df.corr()
display(corr_mat)
sns.heatmap(corr_mat, annot=True)

In [ ]:
# Manual correlation computation
cov_TP = np.cov(df.temp, df.pressure)
print(cov_TP)

corr_m = cov_TP / (np.std(df.temp) * np.std(df.pressure))
print(corr_m)

In [ ]:
# Scatter plot
plt.scatter(df.bwr, df.temp)
plt.xlabel("bwr")
plt.ylabel("temp")

In [ ]:
# Pairplots
sns.pairplot(df)
# sns.pairplot(df[['temp', 'time']])

---
## 2. Linear Regression (Least Squares)

Manual implementation with design matrix, parity plot, confidence intervals

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

In [ ]:
data = r"C:\Users\Student\Downloads\Example_ML.csv"
df = pd.read_csv(data)
df.head()

In [ ]:
X = np.column_stack((df.FA, df.R, df.Age))
n = len(df.FA)

# Design matrix
X_d = np.column_stack((np.ones(n), X))
y = df.Price

In [ ]:
# Least squares solution
beta_hat = np.linalg.inv(X_d.T @ X_d) @ X_d.T @ y
print(beta_hat)

In [ ]:
# Predicted value of house price
y_hat = X_d @ beta_hat
print(y_hat)

In [ ]:
# Residuals and metrics
residual = y - y_hat
SSR = np.sum(residual**2)

# Sum of squared residuals
SST = np.sum((y - np.mean(y))**2)

R2 = 1 - (SSR / SST)
print(R2)

# Mean Absolute Error
MAE = np.mean(abs(residual))
print(MAE)

# Mean Squared Error
MSE = SSR / n
print(MSE)

# Root Mean Squared Error
RMSE = np.sqrt(MSE)
print(RMSE)

In [ ]:
# Parity Plot
plt.scatter(y, y_hat)
plt.plot([y.min(), y.max()], [y_hat.min(), y_hat.max()], 'k--')
plt.xlabel("y_observed")
plt.ylabel("y_predicted")

In [ ]:
# No. of parameters to be estimated
p = 4

# sigma_hat2
sigma2 = SSR / (n - p)

# Covariance matrix of beta
cov_beta = sigma2 * np.linalg.inv(X_d.T @ X_d)
print(cov_beta)

In [ ]:
# Standard errors in estimated beta_hat
se_beta = np.sqrt(np.diag(cov_beta))
print(se_beta)

In [ ]:
# Critical t-score
alpha = 0.05
t_score = stats.t.ppf(1 - alpha/2, n - p)
print(t_score)

In [ ]:
# Estimating the confidence intervals
lb = beta_hat - t_score * se_beta
ub = beta_hat + t_score * se_beta

CI = np.column_stack((lb, beta_hat, ub))
print(CI)

In [ ]:
# Prediction function
def predict(FA, R, Age):
    X_d = np.array([1, FA, R, Age])
    y_pred = X_d @ beta_hat
    return y_pred

y_new = predict(FA=200, R=3, Age=5)
print(y_new)

---
## 3. Ridge Regression (Regularization)

L2 regularization with penalty parameter lambda

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
# No. of observations
n = 100

X = np.linspace(0, 5, n)

# Observed data points
y_true = 3 + 2*X - 0.5*X**2 + 0.1*X**3 - 0.05*X**4

# Observations corrupted with noise (0 Mean and 0.5 Variance)
y = y_true + np.random.normal(0, 0.5, size=n)

X_features = np.column_stack([X, X**2, X**3, X**4])

X_train, X_Test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

mean_train = X_train.mean(axis=0)
std_train = X_train.std(axis=0, ddof=0)

X_train_scaled = (X_train - mean_train) / std_train

In [ ]:
# Normalizing/scaling the input features with 0 mean and unit standard deviation
X_scaled = np.column_stack([X_train_scaled, X_train_scaled**2, X_train_scaled**3, X_train_scaled**4])

# Design matrix for the scaled training inputs
X_d = np.column_stack([np.ones(len(X_scaled)), X_scaled])

# Identity matrix comprising of slope only
I = np.eye(X_d.shape[1])
print(I)

# Penalise the slopes only not the intercept
I[0, 0] = 0
print(I)

In [ ]:
# Penalty parameter
lamda = 1

# Least squares estimates for Ridge Regression
beta_ridge = np.linalg.inv(X_d.T @ X_d + lamda*I) @ X_d.T @ y_train
print(beta_ridge)

# Predict y based on the scaled X_train
y_pred_train = X_d @ beta_ridge

plt.scatter(y_pred_train, y_train)
plt.plot([y_pred_train.min(), y_pred_train.max()], [y_pred_train.min(), y_pred_train.max()], 'k--')

In [ ]:
# Residuals based on the scaled training inputs
residual_train = (y_pred_train - y_train)

SSR_train = np.sum(residual_train**2)
SST_train = np.sum((y_train - np.mean(y_train))**2)

R2_train = 1 - (SSR_train / SST_train)
print(f"R2_train: {R2_train}")

---
## 4. Maximum Likelihood Estimation (MLE)

MLE for Normal and Bernoulli distributions using optimization

In [ ]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
# Generate some observed data (Normal distribution with mu=5, sigma=2)
np.random.seed(42)
data = np.random.normal(5, 2, size=100)

# Negative log-likelihood function for Normal distribution
def neg_log_likelihood(params, data):
    mu, sigma = params[0], params[1]
    if sigma <= 0:  # variance must be positive
        return np.inf
    n = len(data)
    return 0.5 * n * np.log(2 * np.pi * sigma**2) + np.sum((data - mu)**2) / (2 * sigma**2)

# Initial guess (mean of data, std of data)
init_params = [10, 5]

# Minimize negative log-likelihood
result = minimize(neg_log_likelihood, init_params, args=(data,), method='nelder-mead', bounds=[(None, None), (0, None)])
mu_mle, sigma_mle = result.x

print("MLE mean (mu):", mu_mle)
print("MLE variance (sigma^2):", sigma_mle**2)

In [ ]:
# Generate some Bernoulli data (true p=0.7)
np.random.seed(42)

# 10 independent coin toss experiments
data = np.random.binomial(n=1, p=0.7, size=10)

# Negative log-likelihood for Bernoulli
def neg_log_likelihood(p, data):
    if p <= 0 or p >= 1:  # ensure p in (0,1)
        return np.inf
    ll = -np.sum(data * np.log(p) + (1 - data) * np.log(1 - p))
    return ll

# Initial guess of p
init_params = [0.3]

# Optimize p using an optimization solver
res = minimize(neg_log_likelihood, init_params, args=(data,), method='nelder-mead', bounds=[(0, 1)])

p_mle = res.x

print("MLE of p (via optimization):", p_mle)
print("Closed-form MLE of p (sample mean):", np.mean(data))

---
## 5. Principal Component Analysis (PCA)

Manual PCA implementation with eigendecomposition and scree plot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
data = r'C:\Users\Riju\Desktop\HTL_data.csv'
df = pd.read_csv(data)

X = df.iloc[:, 0:4].values
y = df.iloc[:, 4].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Covariance matrix
cov_matrix = np.cov(X_scaled.T)
display(cov_matrix)

# Eigen decomposition
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# Sort eigenvalues (and eigenvectors) in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# Compute explained variance ratio
explained_variance_ratio = eigenvalues / np.sum(eigenvalues)
cumulative_variance = np.cumsum(explained_variance_ratio)

In [ ]:
# Display eigenvalues and explained variance
print("\nEigenvalues (Descending):")
print(eigenvalues)

print("\nExplained Variance Ratio (%):")
for i, var in enumerate(explained_variance_ratio):
    print(f"PC{i+1}: {var*100:.2f}%")

print(f"\nCumulative Variance Explained: {cumulative_variance*100}")

In [ ]:
# Project data onto principal components
Z = X_scaled @ eigenvectors  # principal component scores

# Choose number of components
n_components = 3
Z_reduced = Z[:, :n_components]

In [ ]:
# Plot explained variance (Scree plot)
plt.figure(figsize=(6, 4))
PC_No = [1, 2, 3, 4]
plt.plot(PC_No, eigenvalues)
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalues')
plt.title('Scree Plot')
plt.legend()
plt.tight_layout()
plt.show()

---
## 6. Support Vector Regression (SVR)

SVR with RBF kernel, GridSearchCV for hyperparameter tuning, cross-validation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Load dataset
data = r'C:\Users\Riju\Desktop\HTL_data.csv'
df = pd.read_csv(data)

X = df.iloc[:, 0:4].values
y = df.iloc[:, 4].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
# Pipeline
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR())
])

# L2 regularization
# Smaller C = stronger regularization
param_grid = {
    "svr__kernel": ["rbf"],
    "svr__C": [1, 5, 10, 20, 50, 100],
    "svr__epsilon": [0.01, 0.05, 0.1, 0.2],
    "svr__gamma": ["scale", "auto", 0.1, 0.01]
}

# 5-fold CV
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid search
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring="neg_mean_squared_error",
    cv=cv,
    verbose=1,
    return_train_score=True
)
grid_search.fit(X_train, y_train)

print("\nBest Parameters:", grid_search.best_params_)
print("Best Mean CV Score:", grid_search.best_score_)

In [ ]:
# Best model
best_model = grid_search.best_estimator_

# Predictions
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Metrics function
def metrics(true, pred, label):
    print(f"\n{label} Metrics:")
    print("  RMSE:", np.sqrt(mean_squared_error(true, pred)))
    print("  MAE :", mean_absolute_error(true, pred))
    print("  R²  :", r2_score(true, pred))

metrics(y_train, y_train_pred, "Training")
metrics(y_test, y_test_pred, "Testing")

# Cross-validated predictions
cv_pred = cross_val_predict(best_model, X, y, cv=cv)

print("\nCV Metrics:")
print("  RMSE:", np.sqrt(mean_squared_error(y, cv_pred)))
print("  MAE :", mean_absolute_error(y, cv_pred))
print("  R²  :", r2_score(y, cv_pred))

In [ ]:
# Plot results
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title("Actual vs Predicted (Test)")
plt.xlabel("Actual")
plt.ylabel("Predicted")

plt.figure(figsize=(6, 6))
plt.scatter(y_train, y_train_pred, alpha=0.5)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--')
plt.title("Actual vs Predicted (Train)")
plt.xlabel("Actual")
plt.ylabel("Predicted")

plt.tight_layout()
plt.show()

---
## 7. Support Vector Classification (SVM)

SVM for fault classification with confusion matrix and decision boundary visualization

In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
data = r"E:\MLCourse\MLxChE-main\Week-04-Kernel-Methods\evap_data.csv"
df = pd.read_csv(data)
df.head()

In [ ]:
# Visualize fault data
plt.figure(figsize=(8, 8))

class_0 = df.index[df['Status'] == 'Normal']
class_1 = df.index[df['Status'] == 'Pump_failure']

plt.subplot(3, 1, 1)
plt.plot(class_0, df.iloc[class_0]["L2"], 'b')
plt.plot(class_1, df.iloc[class_1]["L2"], 'r')
plt.grid()
plt.ylabel('L2')

plt.subplot(3, 1, 2)
plt.plot(class_0, df.iloc[class_0]["P2"], 'b')
plt.plot(class_1, df.iloc[class_1]["P2"], 'r')
plt.grid()
plt.ylabel('P2')

plt.subplot(3, 1, 3)
plt.plot(class_0, df.iloc[class_0]["X2"], 'b', label='Normal Operation')
plt.plot(class_1, df.iloc[class_1]["X2"], 'r', label='Pump Failure')
plt.grid()
plt.legend()
plt.ylabel('X2')
plt.xlabel('Sample number')

plt.tight_layout()
plt.show()

In [ ]:
X = df.iloc[:, :3].to_numpy()
y = LabelEncoder().fit_transform(df.iloc[:, -1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [ ]:
# Normalize the data set
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

N_train = len(X_train_scaled)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(X_train_scaled[y_train == 0, 0],
           X_train_scaled[y_train == 0, 1],
           X_train_scaled[y_train == 0, 2],
           c='b', label='Normal operation')

ax.scatter(X_train_scaled[y_train == 1, 0],
           X_train_scaled[y_train == 1, 1],
           X_train_scaled[y_train == 1, 2],
           c='r', label='Pump failure')
ax.set_xlabel('Scaled L2')
ax.set_ylabel('Scaled P2')
ax.set_zlabel('Scaled X2')
plt.legend()
plt.show()

In [ ]:
# Train the SVM with different hyperparameters
box_C = np.array([1, 1, 1, 10, 10, 10, 100, 100, 100])
gamma = np.array([0.1, 1, 2, 0.1, 1, 2, 0.1, 1, 2])

for i in range(len(box_C)):
    print(f"Box Constraint: {box_C[i]}, gamma = {gamma[i]}")
    mdl = SVC(C=box_C[i], gamma=gamma[i]).fit(X_train_scaled, y_train)

    y_pred_train = mdl.predict(X_train_scaled)
    y_pred_test = mdl.predict(X_test_scaled)

    print(f"  Accuracy for training: {mdl.score(X_train_scaled, y_train)}")
    cm_train = confusion_matrix(y_train, y_pred_train)
    print(cm_train)

    print(f"  Accuracy for testing: {mdl.score(X_test_scaled, y_test)}")
    cm_test = confusion_matrix(y_test, y_pred_test)
    print(cm_test)

In [ ]:
# Train using L2 and P2 only (for illustration purposes)
X = df.iloc[:, :2].to_numpy()
y = LabelEncoder().fit_transform(df.iloc[:, -1])

sc = StandardScaler()
X_scaled = sc.fit_transform(X)

plt.scatter(X_scaled[y == 0, 0], X_scaled[y == 0, 1], c='b', label='Normal Operation')
plt.scatter(X_scaled[y == 1, 0], X_scaled[y == 1, 1], c='r', label='Pump failure')
plt.xlabel('Scaled L2')
plt.ylabel('Scaled P2')
plt.legend()
plt.show()

In [ ]:
# Decision boundary visualization
mdl = SVC().fit(X_scaled, y)

plt.figure(figsize=(15, 10))
DecisionBoundaryDisplay.from_estimator(mdl, X_scaled, cmap='bwr', alpha=0.8)
plt.scatter(X_scaled[y == 0, 0], X_scaled[y == 0, 1], c='b', edgecolors="k")
plt.scatter(X_scaled[y == 1, 0], X_scaled[y == 1, 1], c='r', edgecolors="k")
plt.show()

---
## 8. Logistic Regression (Binary Classification)

1D and 2D logistic regression with sigmoid curve and decision boundaries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# 1D Logistic Regression Example
X_train = [1.2, 3.1, 4, 3.8, 2.5, 7, 5.2, 5.5, 6.1, 7.5]
y_train = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

Xs = np.array(X_train).reshape(-1, 1)

# Train logistic regression model
clf = LogisticRegression()
clf.fit(Xs, y_train)

# Generate range of feature values for plotting sigmoid
x_vals = np.linspace(min(Xs)-1, max(Xs)+1, 100).reshape(-1, 1)
y_probs = clf.predict_proba(x_vals)[:, 1]

# Find the decision boundary (where sigma(x)=0.5)
boundary = -clf.intercept_[0] / clf.coef_[0]
print(boundary)

# Plot data and sigmoid curve
plt.figure(figsize=(7, 6))
plt.scatter(Xs, y_train, c=y_train, cmap=plt.cm.bwr, edgecolors='k', s=80)
plt.plot(x_vals, y_probs, color="black", linewidth=2, label="Sigmoid curve")

# Add vertical line for sigma(x) = 0.5
plt.axvline(boundary, color="green", linestyle="--", linewidth=2, label="Decision boundary (σ=0.5)")

plt.title("1D Logistic Regression with Decision Boundary")
plt.xlabel("Feature value")
plt.ylabel("Probability of Class 1")
plt.legend()
plt.show()

print("Decision boundary (x where σ=0.5):", boundary)

In [ ]:
# 2D Logistic Regression Example
n_samples = 200

x1 = np.random.randn(n_samples)
x2 = np.random.randn(n_samples)

# True boundary: y = 1 if x1 + x2 > 0 else 0
y = (x1 + x2 > 0).astype(int)
X = np.column_stack((x1, x2))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Call Logistic Regression for Binary Classification
clf = LogisticRegression()
clf.fit(X_train, y_train)

print("Intercept (β0):", clf.intercept_[0])
print("Coefficients (β1, β2):", clf.coef_[0])

In [ ]:
# Computing the sigmoid function for the given z data
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Take one test point
x_test_point = X_test[0]   # first test sample
print("\nTest point (x1, x2):", x_test_point)

# Linear regression model
z = clf.intercept_[0] + np.dot(clf.coef_[0], x_test_point)

# Predicted probability using sigmoid
y_hat_manual = sigmoid(z)
print("Predicted probability (manual sigmoid):", y_hat_manual)

# Predicted probability using sklearn
y_hat_sklearn = clf.predict_proba([x_test_point])[0, 1]
print("Predicted probability (sklearn):", y_hat_sklearn)

# Predicted class (threshold 0.5)
y_class = clf.predict([x_test_point])[0]
print("Predicted class:", y_class)
print("Actual class:", y_test[0])

In [ ]:
# Plotting the decision boundary
plt.figure(figsize=(7, 5))
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap="bwr", edgecolors="k", s=60, label="Actual")

# Decision boundary grid
x1_min, x1_max = X[:, 0].min()-1, X[:, 0].max()+1
x2_min, x2_max = X[:, 1].min()-1, X[:, 1].max()+1
x1x1, x2x2 = np.meshgrid(np.linspace(x1_min, x1_max, 100),
                         np.linspace(x2_min, x2_max, 100))

Z = clf.predict(np.c_[x1x1.ravel(), x2x2.ravel()])
Z = Z.reshape(x1x1.shape)

plt.contourf(x1x1, x2x2, Z, alpha=0.3, cmap="bwr")
plt.xlabel("x1")
plt.ylabel("x2")
plt.title("Logistic Regression Decision Boundary")
plt.legend()
plt.show()

In [ ]:
# Predictions and evaluation
y_pred = clf.predict(X_test)

# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

---
## 9. Locally Weighted Regression

Non-parametric regression using Gaussian kernel weights

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Generate data
np.random.seed(30)
n = 100

X = np.linspace(-2*np.pi, 2*np.pi, n)
y_true = np.sin(X)
y = y_true + np.random.normal(0, 0.2, size=n)

# Locally Weighted Regression
tau = 0.6  # bandwidth
X_design = np.column_stack((np.ones(n), X))   # design matrix with intercept

X_test = np.linspace(-2*np.pi, 2*np.pi, 100)
y_pred = []

for x in X_test:
    # Gaussian kernel weights
    w = np.exp(- (X - x) ** 2 / (2 * tau ** 2))
    W = np.diag(w)

    # Weighted least squares estimates
    theta = np.linalg.pinv(X_design.T @ W @ X_design) @ (X_design.T @ W @ y)

    # Prediction at x0
    x_vec = np.array([1, x])
    y_pred.append(x_vec @ theta)

y_pred = np.array(y_pred)

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='blue', label="Noisy Data", alpha=0.6)
plt.plot(X_test, y_pred, color='red', label=f"LWR (tau={tau})", linewidth=2)
plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.title("Locally Weighted Regression with Gaussian Kernel")
plt.show()

---
## End of Reference Notebook

This notebook demonstrates:
- Manual implementation of algorithms (not just sklearn)
- Proper mathematical notation and variable naming
- Visualization of results (parity plots, decision boundaries, etc.)
- Evaluation metrics computed manually and with sklearn
- Hyperparameter tuning with GridSearchCV
- Cross-validation workflows
- Code complexity level appropriate for intermediate ML course